# Get started with DownClim

`DownClim` is a simple and easy to use Python package for performing a climate downscaling on future climatic projections. Based on a reference product, and a set of projections from [CMIP6](https://pcmdi.llnl.gov/CMIP6/) and [CORDEX](https://cordex.org/) simulations, `DownClim` will help you to generate a climate projection for your region of interest.

Let's see it in action!

## The `DownClimContext` object 

In [1]:
from __future__ import annotations

import ee

from downclim.dataset.cmip6 import CMIP6Context
from downclim.downclim import (
    DownClimContext,
    define_DownClimContext_from_file,
    generate_DownClimContext_template_file,
)


2025-08-28 23:43:51,451 - downclim - INFO - DownClim starting... Enjoy!
2025-08-28 23:43:51,453 - downclim - INFO - DownClim logging system initialized


In [2]:
your_ee_project_id = "downclim"

In [3]:
# Authenticate to Earth Engine to retrieve CMIP6 data
# ee.Authenticate() if necessary, only need to do this once on your machine
ee.Initialize(opt_url="https://earthengine-highvolume.googleapis.com", project=your_ee_project_id)

The `DownClimContext` object is the main object of the `DownClim` package. It contains all the relevant information necessary to perform your downscaling on your area of interest. 



You can check directly the documentation of the `DownClimContext` object to have an extensive overview of how to define it properly. In the following section, we will see the main fields included in the `DownClimContext` object to perform a downscaling.

### Main fields of the `DownClimContext` object

`DownClimContext` objects contain multiple attributes. The most relevant ones are:

- `aois`: the areas of interest. You can define one or multiple areas of interest, and you have the possibility to define them either as a `str`, a `geopandas.GeoDataFrame` or a simple Python `tuple` of coordinates.
    - `str`: the name provided must be recognized by [GADM](https://gadm.org/) to retrieve the corresponding shapefile.
    - `geopandas.GeoDataFrame` containing the directly shapefile of the area of interest.
    - `tuple` of coordinates `(lon_min, lon_max, lat_min, lat_max, str)` where the last `str` is the name of the area of interest.
  
- `variables`: the variables to downscale. You want to select the climate variables that you want to downscale in your future climate. You must provide them as a list of strings, and the name of the variables have to match the `CMIP6` / `CORDEX` naming convention (see [CMIP6 CMOR Tables](https://github.com/PCMDI/cmip6-cmor-tables)). For example, you can select the following variables: `['tas', 'pr']` for 2m temperature and precipitation. Please note that the variables selected must be available both in the climate simulations and in the reference product.

- `baseline_product`: the reference product used for downscaling. 

- `downscaling_methods`: the downscaling method to use. To downscale your climate data on your area of interest, you might want to use different downscaling methods. So far, `DownClim` only supports the `bias_correction` method, but other methods can be easily adopted and will be available soon.

- `use_cmip6` & `use_cordex`: whether to use CMIP6 or CORDEX simulations. You can select the climate simulation you want to use for your future projections. `CMIP6` projections are available for a large number of models and scenarios, at global scale and usually at a coarse resolution. `CORDEX` simulations are regional projections, much less numerous but at a finer resolution.

- `baseline_periods`, `evaluation_periods`, `projection_periods`: the years to use for the baseline, evaluation and projection periods. To perform a downscaling, you must define the baseline period, evaluation period and projection period. 
  - The baseline period is the period used to calibrate the downscaling method. It is a historical period that must overlap the reference product and the historical climate simulations.
  - The evaluation period is the period that overlaps the reference product and the future climate simulations. It is used to evaluate the calibration of the downscaling method performed on the baseline period. 
  - The projection period is the period for which you want to generate the future climate projections downscaled.


You can create a `DownClimContext` object either directly, or by defining a configuration file in a `.yaml` format.

### `DownClimContext` object creation directly

You can instantiate a `DownClimContext` object directly by providing the necessary information either via the constructor or providing a dictionary filled with the required keys. Not all fields are mandatory, and default values are used for the omitted fields.

In [4]:
DownClimContext_example = DownClimContext(
    aoi="Vanuatu",
    variable=["tas", "pr"],
    historical_period=(1980, 1981),
    evaluation_period=(2017, 2018),
    projection_period=(2099, 2100),
    use_cordex=False,
    use_cmip6=True,
    cmip6_context=CMIP6Context()
)


2025-08-28 23:44:20,664 - downclim.aoi - INFO - Retrieving AOI from Vanuatu
2025-08-28 23:44:20,667 - downclim.aoi - INFO -    AOI given as a string: retrieving from GADM for Vanuatu
2025-08-28 23:44:24,568 - downclim.aoi - INFO -    AOI CRS not defined: setting to EPSG:4326 / WGS84


### `DownClimContext` object creation from a `.yaml` file

You can also define a `DownClimContext` object from a `.yaml` file. You must first create a template `.yaml` file. 

In [3]:
generate_DownClimContext_template_file(output_file = './DownClimContext_example.yaml')

And after filling the sections of the file, your `DownClim` configuration file should look like this:

```yaml
##
## All fields except aoi have default values. If the field is not specified, the default value is used.
####################################################
# aoi
aoi: "Vanuatu"

# variables
variable: ["tas", "pr"] 
time_frequency: "mon" 

# downscaling
downscaling_aggregation: "monthly-mean" 
downscaling_method: "bias_correction" 

# data and simulations
baseline_product: "chelsa2"
use_cordex: false 
use_cmip6: true 
cordex_context: {}
cmip6_context: { source: ["IPSL-CM6A-LR"] }
historical_period: [1980, 1981]
evaluation_period: [2017, 2018]
projection_period: [2099, 2100]
evaluation_product: ["chirps", "chelsa2"]

# internal computation
nb_threads: 4
memory_mb: 4096
chunks: { "time": 1, "lat": 1000, "lon": 1000 }

# directories
output_dir: "results"
tmp_dir: "tmp"
keep_tmp_dir: true

# data access
esgf_credential: "config/esgf_credential.yaml"
```

In [5]:
DownClimContext_example = define_DownClimContext_from_file('./DownClimContext_example.yaml')
DownClimContext_example.model_dump()

2025-08-28 23:44:32,615 - downclim.downclim - INFO - Reading DownClimContext from ./DownClimContext_example.yaml
2025-08-28 23:44:32,622 - downclim.downclim - INFO - Context read successfully from ./DownClimContext_example.yaml
2025-08-28 23:44:32,623 - downclim.aoi - INFO - Retrieving AOI from Vanuatu
2025-08-28 23:44:32,626 - downclim.aoi - INFO -    AOI given as a string: retrieving from GADM for Vanuatu
2025-08-28 23:44:35,270 - downclim.aoi - INFO -    AOI CRS not defined: setting to EPSG:4326 / WGS84


{'aoi': [                                            geometry GID_0   NAME_0
  0  MULTIPOLYGON (((169.7766 -20.2488, 169.7719 -2...   VUT  Vanuatu],
 'variable': ['tas', 'pr'],
 'time_frequency': <Frequency.MONTHLY: 'monthly'>,
 'downscaling_aggregation': <Aggregation.MONTHLY_MEAN: 'monthly-mean'>,
 'baseline_product': <DataProduct.CHELSA: product_name='chelsa', period=(1979, 2018), scale_factor={'pr': 0.1, 'tas': 0.1, 'tasmin': 0.1, 'tasmax': 0.1}, add_offset={'pr': 0, 'tas': -273.15, 'tasmin': -273.15, 'tasmax': -273.15}, lon_lat_names={'lon': 'x', 'lat': 'y'}, url='https://os.zhdk.cloud.switch.ch/chelsav2/GLOBAL'>,
 'evaluation_product': [<DataProduct.CHIRPS: product_name='chirps', period=(1980, 2024), scale_factor={'pr': 1}, add_offset={'pr': 0}, lon_lat_names={'lon': 'lon', 'lat': 'lat'}, url='UCSB-CHG/CHIRPS/DAILY'>,
  <DataProduct.CHELSA: product_name='chelsa', period=(1979, 2018), scale_factor={'pr': 0.1, 'tas': 0.1, 'tasmin': 0.1, 'tasmax': 0.1}, add_offset={'pr': 0, 'tas': -2

## Download required data

Once your `DownClimContext` object is defined correctly, you can download the required data defined in the context. You can do it by using the `download_data` method of the `DownClimContext` object.

In [14]:
DownClimContext_example.download_data()

2025-08-28 16:01:04,162 - downclim.downclim - INFO - Starting data download...
2025-08-28 16:01:04,167 - downclim.downclim - INFO -    Downloading baseline product...
2025-08-28 16:01:04,184 - downclim.downclim - INFO - Downloading baseline product...
2025-08-28 16:01:04,186 - downclim.dataset.chelsa2 - INFO - Downloading CHELSA data...
2025-08-28 16:01:04,188 - downclim.dataset.utils - INFO - Checking output directory...
2025-08-28 16:01:04,192 - downclim.dataset.utils - INFO - Setting output directory: results/chelsa
2025-08-28 16:01:04,193 - downclim.dataset.utils - INFO - Checking output directory...
2025-08-28 16:01:04,194 - downclim.dataset.utils - INFO - Setting output directory: tmp/chelsa
2025-08-28 16:01:04,196 - downclim.aoi - INFO - Retrieving AOI names and bounds
2025-08-28 16:01:04,198 - downclim.dataset.chelsa2 - INFO - Downloading CHELSA data...
2025-08-28 16:01:04,306 - downclim.dataset.chelsa2 - INFO - Getting year "1980" for variables "tas" and areas of interest : "[

/Users/arsouze/miniconda3/envs/downclim/lib/python3.13/site-packages/rioxarray/crs.py:38: RuntimeWarning: CRS EPSG:4362 is deprecated. Its non-deprecated replacement EPSG:4956 will be used instead. To use the original CRS, set the OSR_USE_NON_DEPRECATED configuration option to NO.
  return rasterio.crs.CRS.from_user_input(crs_input)


2025-08-28 16:05:22,552 - downclim.dataset.utils - INFO - Saving chirps grid for Vanuatu...
2025-08-28 16:05:22,568 - downclim.dataset.utils - INFO - Grid saved to results/chirps/chirps_Vanuatu_grid.nc
2025-08-28 16:05:22,571 - downclim.dataset.chelsa2 - INFO - Downloading CHELSA data...
2025-08-28 16:05:22,590 - downclim.dataset.utils - INFO - Checking output directory...
2025-08-28 16:05:22,597 - downclim.dataset.utils - INFO - Setting output directory: results/chelsa
2025-08-28 16:05:22,607 - downclim.dataset.utils - INFO - Checking output directory...
2025-08-28 16:05:22,610 - downclim.dataset.utils - INFO - Setting output directory: tmp/chelsa
2025-08-28 16:05:22,620 - downclim.aoi - INFO - Retrieving AOI names and bounds
2025-08-28 16:05:22,624 - downclim.dataset.chelsa2 - INFO - Downloading CHELSA data...
2025-08-28 16:05:22,752 - downclim.dataset.chelsa2 - INFO - Getting year "2017" for variables "tas" and areas of interest : "['Vanuatu']"
2025-08-28 16:05:22,754 - downclim.dat

## Perform a downscaling

Once your `DownClimContext` object is defined correctly and your data downloaded, you can directly perform a downscaling according to the context.

It gets as simple as: 

In [15]:
DownClimContext_example.run_downscaling()

2025-08-28 16:11:44,426 - downclim.downscale - INFO - Starting downscaling process...
2025-08-28 16:11:44,439 - downclim.dataset.utils - INFO - Checking input directory...
2025-08-28 16:11:44,442 - downclim.dataset.utils - INFO - Checking output directory...
2025-08-28 16:11:44,444 - downclim.dataset.utils - WARNING - Output directory not provided. Using default output directory ./results/downscaled.
2025-08-28 16:11:44,445 - downclim.dataset.utils - INFO - Setting output directory: ./results/downscaled
2025-08-28 16:11:44,449 - downclim.dataset.utils - INFO - Creating output subdirectory: ./results/downscaled/cmip6
2025-08-28 16:11:44,458 - downclim.dataset.utils - INFO - Creating output subdirectory: ./results/downscaled/cordex
2025-08-28 16:11:44,464 - downclim.dataset.utils - INFO - Creating output subdirectory: ./results/downscaled/../regridder
2025-08-28 16:11:44,467 - downclim.aoi - INFO - Retrieving AOI names and bounds
2025-08-28 16:11:44,470 - downclim.downscale - INFO - Chec

In [19]:
DownClimContext_example.run_downscaling(downscaling_grid_file="./results/chirps/chirps_Vanuatu_grid.nc", periods_to_downscale=["evaluation"])

2025-08-28 16:19:30,478 - downclim.downscale - INFO - Starting downscaling process...
2025-08-28 16:19:30,499 - downclim.dataset.utils - INFO - Checking input directory...
2025-08-28 16:19:30,505 - downclim.dataset.utils - INFO - Checking output directory...
2025-08-28 16:19:30,509 - downclim.dataset.utils - WARNING - Output directory not provided. Using default output directory ./results/downscaled.
2025-08-28 16:19:30,514 - downclim.dataset.utils - INFO - Setting output directory: ./results/downscaled
2025-08-28 16:19:30,518 - downclim.dataset.utils - INFO - Creating output subdirectory: ./results/downscaled/cmip6
2025-08-28 16:19:30,519 - downclim.dataset.utils - INFO - Creating output subdirectory: ./results/downscaled/cordex
2025-08-28 16:19:30,523 - downclim.dataset.utils - INFO - Creating output subdirectory: ./results/downscaled/../regridder
2025-08-28 16:19:30,524 - downclim.aoi - INFO - Retrieving AOI names and bounds
2025-08-28 16:19:30,530 - downclim.downscale - INFO - Chec

2025-08-28 16:19:30,647 - downclim.downscale - INFO -        Regridding baseline data on the downscaling grid.
2025-08-28 16:19:30,650 - downclim.dataset.utils - INFO - Checking if regridder file from results/chelsa/chelsa_Vanuatu_grid.nc to downscaling grid ./results/chirps/chirps_Vanuatu_grid.nc already exists
2025-08-28 16:19:30,651 - downclim.dataset.utils - INFO - Could not find regridder file in ./results/downscaled/../regridder/regridder-chelsa_Vanuatu_grid-to-chirps_Vanuatu_grid.nc.
                    Creating a new one. This may take a while...
2025-08-28 16:19:36,765 - downclim.downscale - INFO -    CMIP6 simulations to downscale: {'historical': {'results/cmip6/Vanuatu_cmip6_IPSL_IPSL-CM6A-LR_historical_r1i1p1f1_monthly-mean_1980-01-01_1981-12-31.nc': {'output_dir': 'results/cmip6', 'aoi_n': 'Vanuatu', 'data_product': 'cmip6', 'institute': 'IPSL', 'source': 'IPSL-CM6A-LR', 'experiment': 'historical', 'ensemble': 'r1i1p1f1', 'aggregation': 'monthly-mean', 'tmin': '1980-01-01'

```{note}
We perform 2 downscalings here : one on each grid (*i.e.* `./results/chelsa/chelsa_Vanuatu_grid.nc`, the default value, and `./results/chirps/chirps_Vanuatu_grid.nc`). We do so because the [`evaluation` process](#evaluate-your-downscaling) is requested both on `CHIRPS` and `CHELSA` products, and both on their own original grid.

Another way would have been to perform the downscaling on a common grid (*e.g.* CHELSA grid, as this is the baseline product used for downscaling here, hence the default value), and specify the evaluation grid in the `run_evaluation` method.
```

## Evaluate your downscaling

You can even easily evaluate your downscaling over an evaluation period (as defined [above](#main-fields-of-the-downclimcontext-object)).

In [20]:
DownClimContext_example.run_evaluation()

2025-08-28 16:20:22,807 - downclim.evaluation - INFO - Starting evaluation process...
2025-08-28 16:20:22,825 - downclim.dataset.utils - INFO - Checking input directory...
2025-08-28 16:20:22,832 - downclim.dataset.utils - WARNING - Input directory not provided. Using default input directory ./results/downscaled
2025-08-28 16:20:22,834 - downclim.dataset.utils - INFO - Checking output directory...
2025-08-28 16:20:22,835 - downclim.dataset.utils - WARNING - Output directory not provided. Using default output directory ./results/evaluation.
2025-08-28 16:20:22,837 - downclim.dataset.utils - INFO - Setting output directory: ./results/evaluation
2025-08-28 16:20:22,840 - downclim.dataset.utils - INFO - Creating output subdirectory: ./results/evaluation/cmip6
2025-08-28 16:20:22,843 - downclim.dataset.utils - INFO - Creating output subdirectory: ./results/evaluation/cordex
2025-08-28 16:20:22,850 - downclim.evaluation - INFO -    Start evaluation for AOI: Vanuatu
2025-08-28 16:20:22,852 - 

2025-08-28 16:20:23,167 - downclim.evaluation - INFO -        Looking for downscaled simulations to evaluate...
2025-08-28 16:20:23,171 - downclim.evaluation - WARNING - CMIP6 simulations to downscale not provided. Using all files found in ./results/downscaled/cmip6.
2025-08-28 16:20:23,173 - downclim.evaluation - WARNING - CORDEX simulations to downscale not provided. Using all files found in ./results/downscaled/cordex.
2025-08-28 16:20:23,174 - downclim.evaluation - WARNING - No CORDEX simulations to downscale found.
2025-08-28 16:20:23,175 - downclim.evaluation - WARNING - Files that are evaluated must have the format ./results/downscaled/cordex/Vanuatu_cordex*2017*2018*chirps_Vanuatu_grid.nc.
2025-08-28 16:20:24,853 - downclim.evaluation - INFO - 
                            Evaluating results/downscaled/cmip6/Vanuatu_cmip6_IPSL_IPSL-CM6A-LR_ssp245_r1i1p1f1_monthly-mean_2017-01-01_2018-12-31-downscaled-chelsa_baseline-chirps_Vanuatu_grid.nc against ./results/downscaled/../chirps/V

As `results` has been used as the default output directory, you can find all files created there and your file tree should look like this:

```bash
.
├── aois
│   ├── Vanuatu.cpg
│   ├── Vanuatu.dbf
│   ├── Vanuatu.prj
│   ├── Vanuatu.shp
│   └── Vanuatu.shx
├── chelsa
│   ├── chelsa_Vanuatu_grid.nc
│   ├── Vanuatu_chelsa_monthly-mean_1980-1981.nc
│   └── Vanuatu_chelsa_monthly-mean_2017-2018.nc
├── chirps
│   ├── chirps_Vanuatu_grid.nc
│   └── Vanuatu_chirps_monthly-mean_2017-2018.nc
├── cmip6
│   ├── Vanuatu_cmip6_IPSL_IPSL-CM6A-LR_historical_r1i1p1f1_monthly-mean_1980-01-01_1981-12-31.nc
│   ├── Vanuatu_cmip6_IPSL_IPSL-CM6A-LR_ssp245_r1i1p1f1_monthly-mean_2017-01-01_2018-12-31.nc
│   └── Vanuatu_cmip6_IPSL_IPSL-CM6A-LR_ssp245_r1i1p1f1_monthly-mean_2099-01-01_2100-12-31.nc
├── downscaled
│   ├── cmip6
│   │   ├── Vanuatu_cmip6_IPSL_IPSL-CM6A-LR_historical_r1i1p1f1_monthly-mean_1980-01-01_1981-12-31-chelsa_Vanuatu_grid.nc
│   │   ├── Vanuatu_cmip6_IPSL_IPSL-CM6A-LR_historical_r1i1p1f1_monthly-mean_1980-01-01_1981-12-31-chirps_Vanuatu_grid.nc
│   │   ├── Vanuatu_cmip6_IPSL_IPSL-CM6A-LR_ssp245_r1i1p1f1_monthly-mean_2017-01-01_2018-12-31-downscaled-chelsa_baseline-chelsa_Vanuatu_grid.nc
│   │   ├── Vanuatu_cmip6_IPSL_IPSL-CM6A-LR_ssp245_r1i1p1f1_monthly-mean_2017-01-01_2018-12-31-downscaled-chelsa_baseline-chirps_Vanuatu_grid.nc
│   │   └── Vanuatu_cmip6_IPSL_IPSL-CM6A-LR_ssp245_r1i1p1f1_monthly-mean_2099-01-01_2100-12-31-downscaled-chelsa_baseline-chelsa_Vanuatu_grid.nc
│   └── cordex
├── evaluation
│   ├── cmip6
│   │   ├── Vanuatu_cmip6_IPSL_IPSL-CM6A-LR_ssp245_r1i1p1f1_monthly-mean_2017-01-01_2018-12-31-downscaled-chelsa_baseline-chelsa_Vanuatu_grid_evaluation.nc
│   │   └── Vanuatu_cmip6_IPSL_IPSL-CM6A-LR_ssp245_r1i1p1f1_monthly-mean_2017-01-01_2018-12-31-downscaled-chelsa_baseline-chirps_Vanuatu_grid_evaluation.nc
│   └── cordex
└── regridder
    └── regridder-chelsa_Vanuatu_grid-to-chirps_Vanuatu_grid.nc
```

- `aois` : all files related to the areas of interest (AOIs) 
- `chelsa` : all files related to the CHELSA product. Contains the grid file (`netcdf` with `lon` and `lat` variables), in addition to climatologies for baseline and evaluation periods.
- `chirps` : all files related to the CHIRPS product. Contains the grid file (`netcdf` with `lon` and `lat` variables). As this product is used for evaluation only, it contains only a climatology over the evaluation period.
- `cmip6` : all files related to the CMIP6 product. All CMIP6 simulation files over the AOI matching the `CMIP6Context` defined in the `DownClimContext`.
- `cordex` : all files related to the CORDEX product. All CORDEX simulation files over the AOI matching the `CORDEXContext` defined in the `DownClimContext`. Here, empty as we did not requested any CORDEX simulations.
- `downscaled` : all files `CMIP6` and `CORDEX` downscaled products (over the baseline product). We performed 2 downscaling over 2 different grids, although only for the evaluation period for CHIRPS grid.
- `evaluation` : all files related to the evaluation of the downscaled products. 
- `regridder` : all files related to the regridding process. This process is internal to `DownClim`, but it can save a lot of time to reuse existing regridders among different grids.

## You're all set!

Now you can play with your `DownClimContext` object to explore, download, downscale and evaluate climate data over your areas of interest.

However, if you're looking for a finer grain of control over the dataset, we encourage you to [explore all DownClim functions](examples/all_functions.ipynb).